**1137. N-th Tribonacci Number**

The Tribonacci sequence Tn is defined as follows: 

T0 = 0, T1 = 1, T2 = 1, and Tn+3 = Tn + Tn+1 + Tn+2 for n >= 0.

Given n, return the value of Tn.

https://leetcode.com/problems/n-th-tribonacci-number/

In [2]:
def tribonacci(n: int) -> int:
    trib = [0,1,1]
    if n < 3:
        return trib[n]
    for _ in range(n-2):
        trib = [trib[1], trib[2], sum(trib)]
    return trib[-1]

print(tribonacci(4)) # 4
print(tribonacci(25)) # 1389537

4
1389537


**1222. Queens That Can Attack the King**

On a 0-indexed 8 x 8 chessboard, there can be multiple black queens ad one white king.

You are given a 2D integer array queens where queens[i] = [xQueeni, yQueeni] represents the position of the ith black queen on the chessboard. You are also given an integer array king of length 2 where king = [xKing, yKing] represents the position of the white king.

Return the coordinates of the black queens that can directly attack the king. You may return the answer in any order.

https://leetcode.com/problems/queens-that-can-attack-the-king/

In [5]:
def queensAttacktheKing(queens: list[list[int]], king: list[int]) -> list[list[int]]:
    queens = set(map(tuple, queens))
    directions = [(0,1), (1,1), (1,0), (0,-1), (-1,-1), (-1,0), (1,-1), (-1,1)]
    ans = []
    for x,y in directions:
        r,c = king
        while 0 <= r < 8 and 0 <= c < 8:
            r += x
            c += y
            if (r,c) in queens:
                ans.append([r,c])
                break
    return ans

print(queensAttacktheKing(queens = [[0,1],[1,0],[4,0],[0,4],[3,3],[2,4]], king = [0,0])) # [[0,1],[1,0],[3,3]]
print(queensAttacktheKing(queens = [[0,0],[1,1],[2,2],[3,4],[3,5],[4,4],[4,5]], king = [3,3])) # [[2,2],[3,4],[4,4]]

[[0, 1], [3, 3], [1, 0]]
[[3, 4], [4, 4], [2, 2]]


**1456. Maximum Number of Vowels in a Substring of Given Length**

Given a string s and an integer k, return the maximum number of vowel letters in any substring of s with length k.

Vowel letters in English are 'a', 'e', 'i', 'o', and 'u'.

https://leetcode.com/problems/maximum-number-of-vowels-in-a-substring-of-given-length/

In [8]:
from collections import deque
def maxVowels(s: str, k: int) -> int:
    maximum = 0
    current = 0
    vowels = {"a", "e", "i", "o", "u"}
    q = deque([])
    for c in s:
        if len(q) == k:
            l = q.popleft()
            if l in vowels:
                current -= 1
        q.append(c)
        if c in vowels:
            current += 1
            maximum = max(maximum, current)
    return maximum

print(maxVowels(s = "abciiidef", k = 3)) # 3
print(maxVowels(s = "aeiou", k = 2)) # 2
print(maxVowels(s = "leetcode", k = 3)) # 2

3
2
2


**1498. Number of Subsequences That Satisfy the Given Sum Condition**

You are given an array of integers nums and an integer target.

Return the number of non-empty subsequences of nums such that the sum of the minimum and maximum element on it is less or equal to target. Since the answer may be too large, return it modulo 10^9 + 7.

https://leetcode.com/problems/number-of-subsequences-that-satisfy-the-given-sum-condition/

In [34]:
# quite slow
def numSubseq(nums: list[int], target: int) -> int:
    nums.sort()
    
    sum_target = target - nums[0]
    l = 0
    r = len(nums) - 1
    while l <= r:
        m = (l+r) // 2
        if nums[m] <= sum_target:
            l = m + 1
        else:
            r = m - 1
    
    out = 0

    l = 0
    while l <= r:
        out += 2**(r-l)
        l += 1
        while l < len(nums) and nums[l] + nums[r] > target:
            r -= 1

    return out % (10**9 + 7)

# as slow
def numSubseq(nums: list[int], target: int) -> int:
    nums.sort()

    def binary(l, r, t):
        while l <= r:
            m = (l+r) // 2
            if nums[m] <= t:
                l = m + 1
            else:
                r = m - 1
        return r

    out = 0
    l = 0
    r = binary(0, len(nums)-1, target-nums[0])

    while l <= r:
        out += 2**(r-l)
        l += 1
        if l < len(nums):
            r = binary(l, r, target - nums[l])

    return out % (10**9 + 7)

print(numSubseq(nums = [3,5,6,7], target = 9)) # 4
print(numSubseq(nums = [3,3,6,8], target = 10)) # 6
print(numSubseq(nums = [2,3,3,4,6,7], target = 12)) # 61
print(numSubseq(nums = [5,2,4,1,7,6,8], target = 16)) # 127

4
6
61
127


**1964. Find the Longest Valid Obstacle Course at Each Position**

You want to build some obstacle courses. You are given a 0-indexed integer array obstacles of length n, where obstacles[i] describes the height of the ith obstacle.

For every index i between 0 and n - 1 (inclusive), find the length of the longest obstacle course in obstacles such that:

* You choose any number of obstacles between 0 and i inclusive.
* You must include the ith obstacle in the course.
* You must put the chosen obstacles in the same order as they appear in obstacles.
* Every obstacle (except the first) is taller than or the same height as the obstacle immediately before it.

Return an array ans of length n, where ans[i] is the length of the longest obstacle course for index i as described above.

https://leetcode.com/problems/find-the-longest-valid-obstacle-course-at-each-position/

In [42]:
def longestObstacleCourseAtEachPosition(obstacles: list[int]) -> list[int]:
    ans = []
    stacks = []
    for n in obstacles:
        l = 0
        r = len(stacks)-1
        while l <= r:
            m = (l+r) // 2
            if stacks[m][-1] <= n:
                l = m + 1
            else:
                r = m - 1
        if l == len(stacks):
            stacks.append([n])
        else:
            stacks[l].append(n)
        ans.append(l+1)
    return ans

print(longestObstacleCourseAtEachPosition([1,2,3,2])) # [1,2,3,3]
print(longestObstacleCourseAtEachPosition([2,2,1])) # [1,2,1]
print(longestObstacleCourseAtEachPosition([3,1,5,6,4,2])) # [1,1,2,3,2,2]

[1, 2, 3, 3]
[1, 2, 1]
[1, 1, 2, 3, 2, 2]


**2079. Watering Plants**

You want to water n plants in your garden with a watering can. The plants are arranged in a row and are labeled from 0 to n - 1 from left to right where the ith plant is located at x = i. There is a river at x = -1 that you can refill your watering can at.

Each plant needs a specific amount of water. You will water the plants in the following way:

* Water the plants in order from left to right.
* After watering the current plant, if you do not have enough water to completely water the next plant, return to the river to fully refill the watering can.
* You cannot refill the watering can early.

You are initially at the river (i.e., x = -1). It takes one step to move one unit on the x-axis.

Given a 0-indexed integer array plants of n integers, where plants[i] is the amount of water the ith plant needs, and an integer capacity representing the watering can capacity, return the number of steps needed to water all the plants.

https://leetcode.com/problems/watering-plants/

In [3]:
def wateringPlants(plants: list[int], capacity: int) -> int:
    counter = len(plants)
    current = capacity
    for i in range(len(plants)):
        current -= plants[i]
        if current < 0:
            current = capacity - plants[i]
            counter += 2 * i
    return counter

print(wateringPlants(plants = [2,2,3,3], capacity = 5)) # 14
print(wateringPlants(plants = [1,1,1,4,2,3], capacity = 4)) # 30
print(wateringPlants(plants = [7,7,7,7,7,7,7], capacity = 8)) # 49

14
30
49
